In [1]:
import sympy as sp

# Global symbols
t = sp.symbols('t', real=True)
x = sp.Function('x')(t)
u = sp.Function('u')(t)
lam = sp.Function('lam')(t)
C1, C2 = sp.symbols('C1 C2')  # For general constants

In [2]:
def define_hamiltonian(lagrangian_expr, dynamics_expr):
    return lagrangian_expr + lam * dynamics_expr

def solve_costate(H, t_f=None, lam_tf=None, general=False):
    dHdx = sp.diff(H, x)
    ode = sp.Eq(sp.diff(lam, t), -dHdx)
    if general:
        sol = sp.dsolve(ode, lam)
    else:
        sol = sp.dsolve(ode, lam, ics={lam.subs(t, t_f): lam_tf})
    return sol.rhs

def solve_optimal_control(H, lam_expr):
    dHdu = sp.diff(H, u)
    u_sol = sp.solve(sp.Eq(dHdu, 0), u)[0]
    return u_sol.subs(lam, lam_expr)

def solve_state(f_dyn, u_expr, x0=None, t0=None, general=False):
    f_subbed = f_dyn.subs(u, u_expr)
    ode = sp.Eq(sp.diff(x, t), f_subbed)
    if general:
        sol = sp.dsolve(ode, x)
    else:
        sol = sp.dsolve(ode, x, ics={x.subs(t, t0): x0})
    return sol.rhs

def enforce_boundary_conditions(x_expr, t_0, x_0, t_f, x_f):
    C = list(x_expr.free_symbols & {C1, C2})
    eqs = [
        sp.Eq(x_expr.subs(t, t_0), x_0),
        sp.Eq(x_expr.subs(t, t_f), x_f)
    ]
    sol = sp.solve(eqs, C, dict=True)[0]
    return x_expr.subs(sol)

# ---------------------------- Main Entry Function ----------------------------

def pontryagin_pipeline(L, f_dyn, x_cond, t_int, lam_tf=None):
    """
    Unified Pontryagin pipeline:
    - If x_cond = (x0, xf): uses two-point boundary value solution
    - If x_cond = scalar: uses x0 and lam_tf for single-boundary case
    """
    t_0, t_f = t_int

    # Single or two-point condition?
    is_two_point = isinstance(x_cond, (tuple, list)) and len(x_cond) == 2

    if is_two_point:
        x_0, x_f = x_cond
        H = sp.simplify(define_hamiltonian(L, f_dyn))
        lam_expr = sp.simplify(solve_costate(H, general=True))
        u_expr = sp.simplify(solve_optimal_control(H, lam_expr))
        x_expr_gen = sp.simplify(solve_state(f_dyn, u_expr, general=True))
        x_expr_final = sp.simplify(enforce_boundary_conditions(x_expr_gen, t_0, x_0, t_f, x_f))
        return {
            'Hamiltonian': H,
            'lambda(t)': lam_expr,
            'u(t)': u_expr,
            'x_general(t)': x_expr_gen,
            'x(t)': x_expr_final
        }

    else:
        x_0 = x_cond
        H = sp.simplify(define_hamiltonian(L, f_dyn))
        lam_expr = sp.simplify(solve_costate(H, t_f, lam_tf, general=False))
        u_expr = sp.simplify(solve_optimal_control(H, lam_expr))
        x_expr = sp.simplify(solve_state(f_dyn, u_expr, x_0, t_0, general=False))
        return {
            'Hamiltonian': H,
            'lambda(t)': lam_expr,
            'u(t)': u_expr,
            'x(t)': x_expr
        }

In [3]:
problems = [
    {
        "label": "Problem 1",
        "L_expr": sp.exp(t)*x - u**2,
        "Dyn_expr": u,
        "x_cond": 1,
        "t_int": (0, 2),
        "lam_tf": 0
    },
    {
        "label": "Problem 2",
        "L_expr": x - t*u**2,
        "Dyn_expr": u,
        "x_cond": 0,
        "t_int": (1, 3),
        "lam_tf": 0
    },
    {
        "label": "Problem 3",
        "L_expr": x * sp.cos(t) - u**2,
        "Dyn_expr": u,
        "x_cond": 2,
        "t_int": (0, 1),
        "lam_tf": 0
    },
    {
        "label": "Problem 4",
        "L_expr": x - u**2,
        "Dyn_expr": u,
        "x_cond": (2, 4),
        "t_int": (0, 1),
        "lam_tf": None
    },
    {
        "label": "Problem 5",
        "L_expr": x - u**2,
        "Dyn_expr": u - x,
        "x_cond": (1, sp.E**2),
        "t_int": (0, 2),
        "lam_tf": None
    },
    {
        "label": "Problem 6",
        "L_expr": 3*t*x - u**2,
        "Dyn_expr": u,
        "x_cond": (0, 1),
        "t_int": (0, 1),
        "lam_tf": None
    },
    {
        "label": "Problem 7",
        "L_expr": 5*x - 2*u**2,
        "Dyn_expr": u,
        "x_cond": (2, 4),
        "t_int": (0, 1),
        "lam_tf": None
    },
    {
        "label": "Problem 8",
        "L_expr": 4*t*x - 3*u**2,
        "Dyn_expr": u - x,
        "x_cond": (1, sp.E**2),
        "t_int": (0, 2),
        "lam_tf": None
    },
    {
        "label": "Problem 9",
        "L_expr": t**2 * x - u**2,
        "Dyn_expr": u,
        "x_cond": (0, 1),
        "t_int": (0, 1),
        "lam_tf": None
    }
]

for prob in problems:
    print("="*40)
    print(prob["label"])
    result = pontryagin_pipeline(
        prob["L_expr"], prob["Dyn_expr"], prob["x_cond"], prob["t_int"], prob["lam_tf"]
    )
    for key, val in result.items():
        print(f"{key}: {val}")
    print(sp.N(result['x(t)']))


Problem 1
Hamiltonian: lam(t)*u(t) - u(t)**2 + x(t)*exp(t)
lambda(t): -exp(t) + exp(2)
u(t): -exp(t)/2 + exp(2)/2
x(t): t*exp(2)/2 - exp(t)/2 + 3/2
3.69452804946533*t - 0.5*exp(t) + 1.5
Problem 2
Hamiltonian: -t*u(t)**2 + lam(t)*u(t) + x(t)
lambda(t): 3 - t
u(t): (3 - t)/(2*t)
x(t): -t/2 + 3*log(t)/2 + 1/2
-0.5*t + 1.5*log(t) + 0.5
Problem 3
Hamiltonian: lam(t)*u(t) - u(t)**2 + x(t)*cos(t)
lambda(t): -sin(t) + sin(1)
u(t): -sin(t)/2 + sin(1)/2
x(t): t*sin(1)/2 + cos(t)/2 + 3/2
0.420735492403948*t + 0.5*cos(t) + 1.5
Problem 4
Hamiltonian: lam(t)*u(t) - u(t)**2 + x(t)
lambda(t): C1 - t
u(t): C1/2 - t/2
x_general(t): C1*t/2 + C2 - t**2/4
x(t): -t**2/4 + 9*t/4 + 2
-0.25*t**2 + 2.25*t + 2.0
Problem 5
Hamiltonian: (u(t) - x(t))*lam(t) - u(t)**2 + x(t)
lambda(t): C1*exp(t) + 1
u(t): C1*exp(t)/2 + 1/2
x_general(t): C1*exp(t)/4 + C2*exp(-t) + 1/2
x(t): (exp(2*t) + exp(t) + exp(t + 2) + 2*exp(2*t + 2) - exp(2))*exp(-t)/(2*(1 + exp(2)))
0.0596014610110588*(exp(2*t) + exp(t) + exp(t + 2) + 2.0*exp